# visualizing termination time versus domain size
Tim Tyree<br>
6.11.2025

In [1]:
import os,sys,pandas as pd,numpy as np

In [2]:
from lib import *

# define module

In [3]:
# parse_iolines
def parse_iolines_single(input_fn, printing=False):
    n_input=-9999
    n_output=-9999
    with open(input_fn) as f:
#         trgt1='Printing Inputs...\n'
#         trgt2='Printing Outputs...\n'
        trgt1='Printing Inputs:\n'
        trgt2='Printing Outputs:\n'
        for n,line in enumerate(f):
            if trgt1 == line:
                if printing:
                    print(f'found inputs starting after line {n}')
                n_input=n
            if trgt2 == line:
                if printing:
                    print(f'found outputs starting after line {n}')
                n_output=n
    return n_input,n_output

In [4]:
def parse_outputs_single_trial(input_fn, n_output):
    """
    Example Usage:
TermTime = parse_outputs_single_trial(input_fn, n_output)
    """
    with open(input_fn) as f:
        for n,line in enumerate(f):
            if n == n_output+1:
                lin0 =line
                TermTime = eval(lin0[:-2].split('=')[-1])
                return TermTime
#         if n == n_output+1:
#             line_N = line
#         if n == n_output+2:
#             line_CollTime =line
# N_values=np.array([eval(s) for s in line_N[:-2].split(',')])
# CollTime_values=np.array([eval(s) for s in line_CollTime[:-2].split(',')])
# # CollTime_values=np.array([eval(s) for s in line_CollTime[:-1].split(',')])
# df=pd.DataFrame({
#     'N':N_values,
#     'CollTime':CollTime_values
# })
# df=df.loc[df.CollTime!=-9999]
# df.reset_index(inplace=True,drop=True)
## return df

In [5]:
# parse_outputs(input_fn, n_output=)
def parse_log_single(input_fn,printing=False):
    """
    Example Usage:
dict_data = parse_log_single(input_fn,printing=False)
    """
    n_input,n_output=parse_iolines_single(input_fn, printing=False)
    if n_input==-9999:
        if printing:
            print(f"Warning: n_input not found for input_fn={input_fn}.\nreturning None...")
        return None
    col_name_lst, col_value_lst=parse_inputs(input_fn,n_input,n_output,printing=printing)
    dict_inputs=dict(zip(col_name_lst,col_value_lst))
    TermTime = parse_outputs_single_trial(input_fn, n_output)
    dict_data = dict(dict_inputs)
    dict_data['TermTime'] = TermTime
    return dict_data

# dev

In [6]:
run_num = 48
# fldr = f"{nb_dir}/data/osg_output/run_44"
# fldr = f"{nb_dir}/data/osg_output/run_45"
# fldr = f"{nb_dir}/data/osg_output/run_46"
# fldr = f"{nb_dir}/data/osg_output/run_47"
fldr = f"{nb_dir}/data/osg_output/run_{run_num}"

In [7]:
assert os.path.exists(fldr)

In [8]:
dict_data_lst = []
for fn in os.listdir(fldr):
    if fn.find('.out.')!=-1:
        input_fn = os.path.join(fldr,fn)
        dict_data = parse_log_single(input_fn,printing=False)
        if dict_data is not None:
            #record
            dict_data_lst.append(dict_data)
df=pd.DataFrame(dict_data_lst); del dict_data_lst

In [9]:
# np.array([x for x in dict_data_lst if x is None])

In [10]:
# pd.DataFrame(dict_data_lst)

In [11]:
# len(dict_data_lst)

In [12]:
# input_fn

In [13]:
# !atom '/Users/timothytyree/Documents/GitHub/bgmc/python/data/osg_output/run_45/job.out.17477944.1912'

In [14]:
# dict_data = parse_log_single(input_fn,printing=True)
# dict_data

In [15]:
np.random.seed(42)
printing=True
if printing:
    print(f"LPM with Same Creation (Run #46):")
dict_lst = []
for (D,L),g in df.groupby(by=['D','L']):
    tau_values = g['TermTime'].values
    meanx,Delta_meanx,num_obs,p_normal=comp_mean_bootstrap_uncertainty(tau_values)
    printing=True
    a=g['varkappa'].max()
    chi=g['chi'].max()
    if printing:
        print(f"- {D=}, {L=}: mean tau: {meanx:.4f} +/- {Delta_meanx:.4f} seconds (N={num_obs}, {p_normal=:.4f})")
    dict_lst.append(dict(D=D,L=L,a=a,chi=chi,
                         tau=meanx,Delta_tau=Delta_meanx,
                         num_obs=num_obs,p_normal=p_normal,
                        ))

LPM with Same Creation (Run #46):
- D=0.048, L=3.75: mean tau: 2.4685 +/- 0.1061 seconds (N=2000, p_normal=0.7075)
- D=0.048, L=4.5: mean tau: 17.8213 +/- 8.9452 seconds (N=2000, p_normal=0.0000)
- D=0.048, L=4.75: mean tau: 20.8552 +/- 7.8220 seconds (N=2000, p_normal=0.0000)
- D=0.048, L=5.0: mean tau: 44.0309 +/- 14.5943 seconds (N=2000, p_normal=0.0000)
- D=0.048, L=5.25: mean tau: 68.1883 +/- 15.8884 seconds (N=2000, p_normal=0.0000)
- D=0.048, L=5.625: mean tau: 174.4341 +/- 26.1402 seconds (N=2000, p_normal=0.0002)
- D=0.048, L=6.25: mean tau: 622.5637 +/- 46.9114 seconds (N=2000, p_normal=0.0026)
- D=0.048, L=7.075: mean tau: 2771.8389 +/- 95.9004 seconds (N=2000, p_normal=0.1372)
- D=0.048, L=8.75: mean tau: 5532.8212 +/- 57.0494 seconds (N=2000, p_normal=0.1833)
- D=0.42, L=5.0: mean tau: 1.6145 +/- 2.0866 seconds (N=2000, p_normal=0.0000)
- D=0.42, L=6.25: mean tau: 2.0326 +/- 0.7849 seconds (N=2000, p_normal=0.0000)
- D=0.42, L=7.5: mean tau: 15.9232 +/- 7.0602 seconds (N=2

In [16]:
df_out = pd.DataFrame(dict_lst)
df_out

,D,L,a,chi,tau,Delta_tau,num_obs,p_normal
0,0.048,3.750,1.552,2.939433,2.468460,0.106054,2000,7.074529e-01
1,0.048,4.500,1.552,2.939433,17.821323,8.945163,2000,7.875762e-09
2,0.048,4.750,1.552,2.939433,20.855177,7.822002,2000,5.909675e-20
3,0.048,5.000,1.552,2.939433,44.030900,14.594332,2000,1.045699e-20
4,0.048,5.250,1.552,2.939433,68.188257,15.888418,2000,1.048170e-05
5,0.048,5.625,1.552,2.939433,174.434131,26.140227,2000,1.807385e-04
6,0.048,6.250,1.552,2.939433,622.563696,46.911436,2000,2.633884e-03
7,0.048,7.075,1.552,2.939433,2771.838900,95.900427,2000,1.371983e-01
8,0.048,8.750,1.552,2.939433,5532.821227,57.049441,2000,1.832543e-01
9,0.420,5.000,9.300,2.075269,1.614531,2.086622,2000,1.220362e-24


In [17]:
# savedir = f"{nb_dir}/data/osg_output/run_46_tau_vs_L.csv"
# savedir = f"{nb_dir}/data/osg_output/run_47_tau_vs_L.csv"
savedir = f"{nb_dir}/data/osg_output/run_{run_num}_tau_vs_L.csv"

In [18]:
df_out.to_csv(savedir,index=False)
print(savedir)

/Users/timothytyree/Documents/GitHub/bgmc/python/data/osg_output/run_48_tau_vs_L.csv


In [19]:
beep(3)

-n 
-n 
-n 


0